In [ ]:
import pandas as pd 

In [ ]:
df = pd.read_csv("C:\SINH\☆Work☆\ML\Hands-on ML\DataSet\housing.csv")
print(df.shape)
df.count()

In [ ]:
df.head(10)

In [ ]:
df.info()

## filling the data 

In [ ]:
# df.total_bedrooms.fillna(df.total_bedrooms.median(),inplace=True)

In [ ]:
df.ocean_proximity.value_counts()
# df['ocean_proximity'].value_counts() # both are same i think

In [ ]:
df.describe()

### histograms
will show which values repeted highly.
insite about value repetation

In [ ]:
# jupyter own backend,so it'll render within
%matplotlib inline  

import matplotlib.pyplot as plt
# df.median_income.hist(color = "red",bins = 100,figsize=(20,15),lw=0)
# plt.show()  # this is optional

# Creating a test set function 

In [ ]:
# import numpy as np

# def split_train_test(data,test_ratio):
#     np.random.seed(42) #it always generates the same shuffled indices
#     shuffled_indices = np.random.permutation(len(data))
#     test_set_size = int(len(data) * test_ratio)
#     test_indices = shuffled_indices[:test_set_size]
#     train_indices = shuffled_indices[test_set_size:]
#     return data.iloc[train_indices],data.iloc[test_indices]

In [ ]:
# train_set , test_set = split_train_test(df,0.2)
# len(train_set),len(test_set)

## With the help of sklearn

In [ ]:
from sklearn.model_selection import train_test_split

train_set,test_set = train_test_split(df,test_size=0.2,random_state=42)
len(train_set),len(test_set)

## stratified sampling
### first need to create an income category attribute based on median_income

In [ ]:
import numpy as np
df['income_cate'] = pd.cut(df.median_income,bins=[0.,1.5,3.0,4.5,6., np.inf],labels=[1,2,3,4,5])
# df.income_cate.head(10)

In [ ]:
# df.income_cate.hist()
# plt.show()

### Now you are ready to do stratified sampling based on the income category. Use Scikit-Learn’s StratifiedShuffleSplit class

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index ,test_index in split.split(df,df.income_cate):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]    
    
# strat_test_set.income_cate.value_counts() / len(strat_test_set)


### remove the income_cat attribute so the data is back to its original state

In [ ]:
for set_ in (strat_train_set,strat_test_set):
    set_.drop("income_cate",axis=1,inplace=True)

#### Let’s create a copy so you can play with it without harming the training set

In [ ]:
df1 =  strat_train_set.copy() 
df1.count()

## Visualizing Geographical Data
#### Since there is geographical information (latitude and longitude), it is a good idea to create a scatterplot of all districts to visualize the data 

In [ ]:
# df1.plot(kind='scatter',x='longitude',y='latitude',alpha=0.1)
# plt.show()

In [ ]:
# df1.plot(kind='scatter',x = 'longitude', y ='latitude',alpha=0.4,
#         s=df1.population/100,label='population',figsize=(10,7),
#         c='median_house_value',cmap=plt.get_cmap('jet'),colorbar=True,)
# plt.show()

## Correlations
#### Since the dataset is not too large, we can easily compute the standard correlation coeffcient (also called Pearson’s r) between every pair of attributes using the corr() method:

In [ ]:
corr_matrix = df1.corr()

In [ ]:
corr_matrix.median_house_value.sort_values(ascending=False)

### Another way to check for correlation between attributes is to use Pandas’ "scatter_matrix function".

In [ ]:
from pandas.plotting import scatter_matrix
attributes = ["median_house_value", "median_income", "total_rooms","housing_median_age"]
# scatter_matrix(df1[attributes],figsize=(12,8))
# plt.show()

In [ ]:
# df1.plot(kind='scatter',x = 'median_income',y='median_house_value',alpha=0.1)

### Adding new attributes 
#### 1."rooms_per_household"
#### 2."bedrooms_per_room"
#### 3."population_per_household"

In [ ]:
df1['rooms_per_household'] = df1.total_rooms/df1.households
df1['bedrooms_per_room'] = df1.total_bedrooms/df1.total_rooms
df1['population_per_household'] = df1.population/df1.households


In [ ]:
corr_matrix = df1.corr()
corr_matrix.median_house_value.sort_values(ascending=False)

## Prepare the Data for ML Algo 

In [ ]:
df1 = strat_train_set.drop('median_house_value',axis=1)
df1_labels = strat_train_set.median_house_value.copy()


## Data cleaning

#### 1 Get rid of the corresponding districts......   or
#### 2 Get rid of the whole attribute......   or 
#### 3 Set the values to some value (zero, the mean, the median, mode,etc.).

In [ ]:
# df1.dropna(subset=['total_bedrooms']) # option 1
# df1.drop("total_bedrooms",axis=1)     # option 2
# median = df1.total_bedrooms.median()    # option 3
# df1.total_bedrooms.fillna(median,inplace=True)

### with help of sklearn SimpleImputer

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')


<!-- Since the median can only be computed on numerical attributes, we need to create a copy of the data without the text attribute "ocean_proximity". -->

Since the median can only be computed on numerical attributes, we need to create a copy of the data without the text attribute "ocean_proximity".

In [ ]:
df_num = df1.drop("ocean_proximity",axis=1)
imputer.fit(df_num)
df_num.count()

In [ ]:
imputer.statistics_

In [ ]:
df_num.median().values

## Handling Text and Categorical Attributes